In [11]:
import os
import numpy as np
import pandas as pd
from reconciliation_utils import *

In [12]:
estimator = "DeepAR"

test_ids = np.empty((0, ))
test_labels = np.empty((0, 28))
test_forecasts = np.empty((0, 28))

for level in range(1, 13):
    level_dir = f"../previous_result/level {level}"
    for root, dirs, files in os.walk(level_dir):
        for file in files:
            if file.startswith('test_labels'):
                with open(os.path.join(root, file), 'rb') as pickle_file:
                    datas = pd.read_pickle(pickle_file)
                    arrays = np.array(datas)
                    arrays = np.squeeze(arrays, axis=-1)
                    arrays = arrays[:, -28:]
                    test_labels = np.concatenate((test_labels, arrays), axis=0)
            if file.startswith('test_forecasts'):
                with open(os.path.join(root, file), 'rb') as pickle_file:
                    datas = pd.read_pickle(pickle_file)
                    ids = np.array([data.item_id for data in datas])
                    test_ids = np.concatenate((test_ids, ids), axis=0)
                    arrays = np.array([data.quantile(0.5) for data in datas])
                    test_forecasts = np.concatenate((test_forecasts, arrays), axis=0)

In [13]:
node_mapping = {test_id: test_idx for test_idx, test_id in enumerate(test_ids)}
node_info = {test_id: parse_id_components(test_id) for test_id in test_ids}

In [14]:
node_mapping

{'total': 0,
 'CA': 1,
 'TX': 2,
 'WI': 3,
 'CA_1': 4,
 'CA_2': 5,
 'CA_3': 6,
 'CA_4': 7,
 'TX_1': 8,
 'TX_2': 9,
 'TX_3': 10,
 'WI_1': 11,
 'WI_2': 12,
 'WI_3': 13,
 'FOODS': 14,
 'HOBBIES': 15,
 'HOUSEHOLD': 16,
 'FOODS_1': 17,
 'FOODS_2': 18,
 'FOODS_3': 19,
 'HOBBIES_1': 20,
 'HOBBIES_2': 21,
 'HOUSEHOLD_1': 22,
 'HOUSEHOLD_2': 23,
 'CA_FOODS': 24,
 'CA_HOBBIES': 25,
 'CA_HOUSEHOLD': 26,
 'TX_FOODS': 27,
 'TX_HOBBIES': 28,
 'TX_HOUSEHOLD': 29,
 'WI_FOODS': 30,
 'WI_HOBBIES': 31,
 'WI_HOUSEHOLD': 32,
 'CA_FOODS_1': 33,
 'CA_FOODS_2': 34,
 'CA_FOODS_3': 35,
 'CA_HOBBIES_1': 36,
 'CA_HOBBIES_2': 37,
 'CA_HOUSEHOLD_1': 38,
 'CA_HOUSEHOLD_2': 39,
 'TX_FOODS_1': 40,
 'TX_FOODS_2': 41,
 'TX_FOODS_3': 42,
 'TX_HOBBIES_1': 43,
 'TX_HOBBIES_2': 44,
 'TX_HOUSEHOLD_1': 45,
 'TX_HOUSEHOLD_2': 46,
 'WI_FOODS_1': 47,
 'WI_FOODS_2': 48,
 'WI_FOODS_3': 49,
 'WI_HOBBIES_1': 50,
 'WI_HOBBIES_2': 51,
 'WI_HOUSEHOLD_1': 52,
 'WI_HOUSEHOLD_2': 53,
 'CA_1_FOODS': 54,
 'CA_1_HOBBIES': 55,
 'CA_1_HOUSEHOL

In [15]:
node_info

{'total': {'level': 1},
 'CA': {'level': 2, 'state': 'CA'},
 'TX': {'level': 2, 'state': 'TX'},
 'WI': {'level': 2, 'state': 'WI'},
 'CA_1': {'level': 3, 'state': 'CA', 'store': '1'},
 'CA_2': {'level': 3, 'state': 'CA', 'store': '2'},
 'CA_3': {'level': 3, 'state': 'CA', 'store': '3'},
 'CA_4': {'level': 3, 'state': 'CA', 'store': '4'},
 'TX_1': {'level': 3, 'state': 'TX', 'store': '1'},
 'TX_2': {'level': 3, 'state': 'TX', 'store': '2'},
 'TX_3': {'level': 3, 'state': 'TX', 'store': '3'},
 'WI_1': {'level': 3, 'state': 'WI', 'store': '1'},
 'WI_2': {'level': 3, 'state': 'WI', 'store': '2'},
 'WI_3': {'level': 3, 'state': 'WI', 'store': '3'},
 'FOODS': {'level': 4, 'category': 'FOODS'},
 'HOBBIES': {'level': 4, 'category': 'HOBBIES'},
 'HOUSEHOLD': {'level': 4, 'category': 'HOUSEHOLD'},
 'FOODS_1': {'level': 5, 'category': 'FOODS', 'dept': '1'},
 'FOODS_2': {'level': 5, 'category': 'FOODS', 'dept': '2'},
 'FOODS_3': {'level': 5, 'category': 'FOODS', 'dept': '3'},
 'HOBBIES_1': {'level

In [16]:
             

edges, edge_types = create_edges(node_mapping)
   
# 4. 결과 저장
# np.save('edges.npy', edges)
# np.save('edge_types.npy', edge_types)
   
# with open('node_mapping.pkl', 'wb') as f:
#     pickle.dump(node_mapping, f)
   
# 5. 통계 출력
print(f"Total nodes: {len(node_mapping)}")
print(f"Total edges: {len(edges)}")
print(f"Up edges: {np.sum(edge_types == 0)}")
print(f"Down edges: {np.sum(edge_types == 1)}")
print(f"Cross edges: {np.sum(edge_types == 2)}")

Total nodes: 42840
Total edges: 5368
Up edges: 2554
Down edges: 2554
Cross edges: 260


In [ ]:
import torch
import numpy as np

def prepare_data_for_gnn(forecasts, edges, edge_types, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    NumPy 배열을 PyTorch Tensor로 변환하여 GNN 모델에 입력할 수 있도록 준비
    
    Args:
        forecasts: numpy 배열, 각 노드의 예측값 [num_nodes, num_timesteps]
        edges: numpy 배열, 엣지 리스트 [num_edges, 2]
        edge_types: numpy 배열, 엣지 타입 [num_edges]
        device: 사용할 디바이스
        
    Returns:
        forecasts_tensor: PyTorch Tensor
        edges_tensor: PyTorch Tensor (Long)
        edge_types_tensor: PyTorch Tensor (Long)
    """
    # 데이터 타입 확인 및 변환
    if isinstance(forecasts, np.ndarray):
        forecasts_tensor = torch.FloatTensor(forecasts).to(device)
    elif isinstance(forecasts, torch.Tensor):
        forecasts_tensor = forecasts.float().to(device)
    else:
        raise TypeError("forecasts must be numpy array or torch tensor")
        
    if isinstance(edges, np.ndarray):
        edges_tensor = torch.LongTensor(edges).to(device)
    elif isinstance(edges, torch.Tensor):
        edges_tensor = edges.long().to(device)
    else:
        raise TypeError("edges must be numpy array or torch tensor")
        
    if isinstance(edge_types, np.ndarray):
        edge_types_tensor = torch.LongTensor(edge_types).to(device)
    elif isinstance(edge_types, torch.Tensor):
        edge_types_tensor = edge_types.long().to(device)
    else:
        raise TypeError("edge_types must be numpy array or torch tensor")
        
    return forecasts_tensor, edges_tensor, edge_types_tensor

def create_train_val_split(forecasts, targets, edges, edge_types, val_ratio=0.2, random_state=42):
    """
    학습 및 검증 데이터셋 분할
    
    Args:
        forecasts: 예측값 텐서
        targets: 실제값 텐서 
        edges: 엣지 텐서
        edge_types: 엣지 타입 텐서
        val_ratio: 검증 세트 비율
        random_state: 랜덤 시드
        
    Returns:
        train_forecasts, val_forecasts: 학습/검증용 예측값
        train_targets, val_targets: 학습/검증용 실제값
    """
    # 재현성을 위한 시드 설정
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    
    # 노드 수
    num_nodes = forecasts.shape[0]
    
    # 노드 인덱스 섞기
    indices = np.random.permutation(num_nodes)
    split_idx = int(num_nodes * (1 - val_ratio))
    
    train_indices = indices[:split_idx]
    val_indices = indices[split_idx:]
    
    # 학습/검증 데이터 분할
    train_forecasts = forecasts[train_indices]
    val_forecasts = forecasts[val_indices]
    
    train_targets = targets[train_indices]
    val_targets = targets[val_indices]
    
    return train_forecasts, val_forecasts, train_targets, val_targets, edges, edge_types

# 사용 예시:
"""
# 데이터 준비
forecasts_tensor, edges_tensor, edge_types_tensor = prepare_data_for_gnn(
    forecasts=test_forecasts,
    edges=edges,
    edge_types=edge_types
)

# 학습/검증 분할
train_forecasts, val_forecasts, train_targets, val_targets, edges, edge_types = create_train_val_split(
    forecasts=forecasts_tensor,
    targets=targets_tensor,
    edges=edges_tensor,
    edge_types=edge_types_tensor
)

# 모델 학습
model = GATModel(input_dim=train_forecasts.shape[1])
trainer = Trainer(model)
history = trainer.train(
    train_forecasts, train_targets,
    val_forecasts, val_targets,
    edges, edge_types
)
"""

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class MessagePassing(nn.Module):
    def __init__(self, input_dim=28, hidden_dim=64, num_heads=4):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        
        # Multi-head attention layers
        self.query = nn.Linear(input_dim, hidden_dim)
        self.key = nn.Linear(input_dim, hidden_dim)
        self.value = nn.Linear(input_dim, hidden_dim)
        
        # Edge type specific transformations
        self.edge_type_embeddings = nn.Embedding(3, hidden_dim)  # 3 types: up, down, cross
        
        # Output projection
        self.output_projection = nn.Linear(hidden_dim, input_dim)
        
    def forward(self, node_features, edges, edge_types, num_nodes=None):
        # 입력 데이터를 텐서로 변환
        device = self.query.weight.device
        
        if isinstance(node_features, np.ndarray):
            node_features = torch.FloatTensor(node_features).to(device)
        
        if isinstance(edges, np.ndarray):
            edges = torch.LongTensor(edges).to(device)
            
        if isinstance(edge_types, np.ndarray):
            edge_types = torch.LongTensor(edge_types).to(device)
            
        if num_nodes is None:
            num_nodes = node_features.shape[0]
        
        # Prepare attention inputs
        Q = self.query(node_features).view(-1, self.num_heads, self.head_dim)
        K = self.key(node_features).view(-1, self.num_heads, self.head_dim)
        V = self.value(node_features).view(-1, self.num_heads, self.head_dim)
        
        # Initialize output
        messages = torch.zeros((num_nodes, self.input_dim), device=device)
        
        # Process each edge type
        for edge_type in range(3):
            mask = edge_types == edge_type
            if not mask.any():
                continue
                
            type_edges = edges[mask]
            edge_embedding = self.edge_type_embeddings(torch.tensor(edge_type, device=device))
            
            # Source and target node features
            source_nodes = type_edges[:, 0]
            target_nodes = type_edges[:, 1]
            
            # Compute attention scores
            scores = torch.matmul(Q[source_nodes], K[target_nodes].transpose(-2, -1))
            scores = scores / torch.sqrt(torch.tensor(self.head_dim, dtype=torch.float, device=device))
            attention = F.softmax(scores, dim=-1)
            
            # Apply attention
            msg = torch.matmul(attention, V[target_nodes])
            msg = msg.reshape(-1, self.hidden_dim)
            
            # Add edge type information
            msg = msg + edge_embedding
            
            # Aggregate messages
            for i, src_idx in enumerate(source_nodes):
                messages[src_idx] += self.output_projection(msg[i])
        
        return messages

class GATModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, num_layers=3, num_heads=4):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # Input projection
        self.input_projection = nn.Linear(input_dim, hidden_dim)
        
        # GAT layers
        self.layers = nn.ModuleList([
            MessagePassing(
                hidden_dim if i > 0 else input_dim,
                hidden_dim,
                num_heads=num_heads
            ) for i in range(num_layers)
        ])
        
        # Layer normalization
        self.layer_norms = nn.ModuleList([
            nn.LayerNorm(hidden_dim) for _ in range(num_layers)
        ])
        
        # Output projection
        self.output_projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
        
    def forward(self, x, edges, edge_types):
        # 데이터 자동 변환
        device = self.input_projection.weight.device
        
        if isinstance(x, np.ndarray):
            x = torch.FloatTensor(x).to(device)
            
        # Store original input for residual connection
        original_x = x
        
        # Initial projection
        hidden = self.input_projection(x)
        
        # Message passing layers
        for layer, layer_norm in zip(self.layers, self.layer_norms):
            # Apply message passing
            message = layer(hidden, edges, edge_types, x.size(0))
            # Residual connection and normalization
            hidden = layer_norm(hidden + message)
            hidden = F.relu(hidden)
        
        # Final projection and residual connection
        adjustment = self.output_projection(hidden)
        return original_x + adjustment

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
import os

# 앞서 정의한 모듈들 import 가정
# from model import MessagePassing, GATModel
# from trainer import Trainer
# from data_preprocessing import prepare_data_for_gnn, create_train_val_split

def load_data(path_to_forecasts, path_to_targets, path_to_edges=None, path_to_edge_types=None):
    """
    시계열 예측값과 실제값, 그래프 구조 로드
    """
    # 예측값 로드
    forecasts = np.load(path_to_forecasts)
    
    # 실제값 로드 (있는 경우)
    if os.path.exists(path_to_targets):
        targets = np.load(path_to_targets)
    else:
        targets = None
    
    # 그래프 구조 로드
    if path_to_edges is not None and path_to_edge_types is not None:
        edges = np.load(path_to_edges)
        edge_types = np.load(path_to_edge_types)
    else:
        # 기본 node_mapping으로부터 edges와 edge_types 생성
        with open('node_mapping.pkl', 'rb') as f:
            node_mapping = pickle.load(f)
        edges, edge_types = create_edges(node_mapping)
    
    return forecasts, targets, edges, edge_types

def run_experiment(forecasts, targets, edges, edge_types, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    전체 실험 과정 실행
    """
    print(f"Running experiment on device: {device}")
    print(f"Data shapes - Forecasts: {forecasts.shape}, Targets: {targets.shape}")
    print(f"Graph statistics - Nodes: {forecasts.shape[0]}, Edges: {edges.shape[0]}")
    
    # 데이터 준비
    forecasts_tensor, edges_tensor, edge_types_tensor = prepare_data_for_gnn(
        forecasts=forecasts,
        edges=edges,
        edge_types=edge_types,
        device=device
    )
    
    if targets is not None:
        targets_tensor = torch.FloatTensor(targets).to(device)
        
        # 학습/검증 분할
        train_forecasts, val_forecasts, train_targets, val_targets, edges_tensor, edge_types_tensor = create_train_val_split(
            forecasts=forecasts_tensor,
            targets=targets_tensor,
            edges=edges_tensor,
            edge_types=edge_types_tensor
        )
        
        print(f"Train/Val split - Train: {train_forecasts.shape[0]}, Validation: {val_forecasts.shape[0]}")
    else:
        # 타겟이 없으면 예측만 수행
        train_forecasts = forecasts_tensor
        train_targets = forecasts_tensor  # 의미없는 값, 사용되지 않음
        val_forecasts = forecasts_tensor
        val_targets = forecasts_tensor
    
    # 모델 초기화
    model = GATModel(input_dim=forecasts.shape[1], hidden_dim=256, num_layers=3)
    trainer = Trainer(model, device=device)
    
    # 모델 학습
    history = trainer.train(
        train_forecasts, train_targets,
        val_forecasts, val_targets,
        edges_tensor, edge_types_tensor,
        epochs=100,
        patience=10
    )
    
    # Ablation study
    if targets is not None:
        print("Running ablation study...")
        ablation_results = ablation_study(
            forecasts_tensor, targets_tensor,
            edges_tensor, edge_types_tensor,
            device=device
        )
        
        # 결과 시각화
        visualize_results(history, ablation_results)
    
    # 최종 예측
    model.load_state_dict(torch.load('best_model.pt'))
    model.eval()
    with torch.no_grad():
        final_predictions = model(forecasts_tensor, edges_tensor, edge_types_tensor)
    
    return final_predictions.cpu().numpy(), history

def visualize_forecasts(original_forecasts, adjusted_forecasts, targets=None, sample_indices=None):
    """
    원본 예측값과 GNN으로 조정된 예측값, 실제값 비교 시각화
    """
    if sample_indices is None:
        # 랜덤 샘플 선택
        sample_indices = np.random.choice(original_forecasts.shape[0], 5, replace=False)
    
    n_samples = len(sample_indices)
    fig, axes = plt.subplots(n_samples, 1, figsize=(15, 4*n_samples))
    
    for i, idx in enumerate(sample_indices):
        ax = axes[i] if n_samples > 1 else axes
        x = np.arange(original_forecasts.shape[1])
        
        # 원본 예측
        ax.plot(x, original_forecasts[idx], 'b-', label='Original Forecast')
        
        # GNN 조정 예측
        ax.plot(x, adjusted_forecasts[idx], 'r-', label='GNN Adjusted Forecast')
        
        # 실제값 (있는 경우)
        if targets is not None:
            ax.plot(x, targets[idx], 'g--', label='Actual Values')
            
        ax.set_title(f'Node {idx} Forecast Comparison')
        ax.set_xlabel('Time')
        ax.set_ylabel('Value')
        ax.legend()
        
        # RMSE 계산 (실제값이 있는 경우)
        if targets is not None:
            original_rmse = np.sqrt(mean_squared_error(targets[idx], original_forecasts[idx]))
            adjusted_rmse = np.sqrt(mean_squared_error(targets[idx], adjusted_forecasts[idx]))
            ax.text(0.05, 0.95, f'Original RMSE: {original_rmse:.4f}\nAdjusted RMSE: {adjusted_rmse:.4f}',
                   transform=ax.transAxes, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()

def ablation_study(features, targets, edges, edge_types, device):
    """확장된 ablation study"""
    results = {}
    metrics = {}
    
    # Baseline: All edge types
    model_all = GATModel(input_dim=features.shape[1])
    trainer_all = Trainer(model_all, device=device)
    history_all = trainer_all.train(features, targets, features, targets, edges, edge_types)
    
    # 전체 모델 성능
    results['all_edges'] = history_all['val_rmse'][-1]
    metrics['all_edges'] = {
        'rmse': history_all['val_rmse'][-1],
        'mae': history_all['val_mae'][-1]
    }
    
    # No GNN baseline (원본 예측 그대로 사용)
    no_gnn_mse = mean_squared_error(targets.cpu().numpy(), features.cpu().numpy())
    no_gnn_mae = mean_absolute_error(targets.cpu().numpy(), features.cpu().numpy())
    results['no_gnn'] = np.sqrt(no_gnn_mse)
    metrics['no_gnn'] = {
        'rmse': np.sqrt(no_gnn_mse),
        'mae': no_gnn_mae
    }
    
    # Remove each edge type one at a time
    for edge_type in range(3):
        edge_type_name = ['up', 'down', 'cross'][edge_type]
        
        # Create mask for selected edges
        mask = edge_types != edge_type
        selected_edges = edges[mask]
        selected_types = edge_types[mask]
        
        # Train model
        model = GATModel(input_dim=features.shape[1])
        trainer = Trainer(model, device=device)
        history = trainer.train(features, targets, features, targets, selected_edges, selected_types)
        
        results[f'without_{edge_type_name}'] = history['val_rmse'][-1]
        metrics[f'without_{edge_type_name}'] = {
            'rmse': history['val_rmse'][-1],
            'mae': history['val_mae'][-1]
        }
    
    # Only one edge type at a time
    for edge_type in range(3):
        edge_type_name = ['up', 'down', 'cross'][edge_type]
        
        # Create mask for selected edges
        mask = edge_types == edge_type
        selected_edges = edges[mask]
        selected_types = edge_types[mask]
        
        # Train model
        model = GATModel(input_dim=features.shape[1])
        trainer = Trainer(model, device=device)
        history = trainer.train(features, targets, features, targets, selected_edges, selected_types)
        
        results[f'only_{edge_type_name}'] = history['val_rmse'][-1]
        metrics[f'only_{edge_type_name}'] = {
            'rmse': history['val_rmse'][-1],
            'mae': history['val_mae'][-1]
        }
    
    # 상세 결과 저장
    with open('ablation_results.pkl', 'wb') as f:
        pickle.dump(metrics, f)
    
    return results

def visualize_results(history, ablation_results):
    """향상된 결과 시각화"""
    # 학습 과정 시각화
    plt.figure(figsize=(18, 12))
    
    # Loss curves
    plt.subplot(2, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.title('Training Loss History')
    plt.legend()
    
    # RMSE curves
    plt.subplot(2, 2, 2)
    plt.plot(history['val_rmse'], label='Validation RMSE')
    plt.xlabel('Epoch')
    plt.ylabel('RMSE')
    plt.title('Validation RMSE History')
    plt.legend()
    
    # Ablation study - RMSE
    plt.subplot(2, 2, 3)
    types = list(ablation_results.keys())
    values = list(ablation_results.values())
    
    # 그래프 x축 레이블 가독성 향상
    plt.barh(types, values)
    plt.xlabel('RMSE')
    plt.title('Ablation Study: Impact of Edge Types on RMSE')
    
    # Performance improvement
    plt.subplot(2, 2, 4)
    if 'no_gnn' in ablation_results and 'all_edges' in ablation_results:
        improvement = 100 * (ablation_results['no_gnn'] - ablation_results['all_edges']) / ablation_results['no_gnn']
        plt.bar(['GNN Improvement'], [improvement])
        plt.ylabel('Error Reduction (%)')
        plt.title(f'RMSE Reduction: {improvement:.2f}%')
    
    plt.tight_layout()
    plt.savefig('training_results.png', dpi=300)
    plt.show()

# 메인 함수
def main():
    # 데이터 로드
    forecasts, targets, edges, edge_types = load_data(
        path_to_forecasts='forecasts.npy',
        path_to_targets='targets.npy',
        path_to_edges='edges.npy',
        path_to_edge_types='edge_types.npy'
    )
    
    # 실험 실행
    adjusted_forecasts, history = run_experiment(forecasts, targets, edges, edge_types)
    
    # 결과 저장
    np.save('adjusted_forecasts.npy', adjusted_forecasts)
    
    # 시각화
    sample_indices = np.random.choice(forecasts.shape[0], 5, replace=False)
    visualize_forecasts(forecasts, adjusted_forecasts, targets, sample_indices)
    
    print("Experiment completed!")

if __name__ == "__main__":
    main()

In [17]:
len(test_forecasts)

42840

In [ ]:
import numpy as np
import pandas as pd


levels = [1, 3, 10, 3, 7, 9, 21, 30, 70, 3049, 9147, 30490]

start_idx = 0
for i, level in enumerate(levels, start=1):
    end_idx = start_idx + level
    
    test_id_level = test_ids[start_idx:end_idx]
    df = pd.DataFrame(test_id_level, columns=['test_id'])
    df.to_csv(f'test_id_level_{i}.csv', index=False)
    
    start_idx = end_idx

In [ ]:
def calculate_wrmsse(y_true, y_pred):
    sales = pd.read_csv('../data/original/sales_train_validation.csv')
    sell_prices = pd.read_csv('../data/original/sell_prices.csv')

    sales = sales.iloc[:, 6:].values

    sell_prices['id'] = sell_prices['store_id'] + '_' + sell_prices['item_id']
    sell_prices = sell_prices[sell_prices['wm_yr_wk'] <= 11613]
    sell_prices = sell_prices.pivot(index='id', columns='wm_yr_wk', values='sell_price')
    sell_prices = sell_prices.values

    N, h = y_true.shape 
    w = sell_prices.shape[1]  

    daily_prices = np.repeat(sell_prices, repeats=7, axis=1)[:, -sales.shape[1]:]
    daily_prices = np.where(np.isnan(daily_prices), np.nan, daily_prices)
    
    squared_errors = np.mean((y_true - y_pred) ** 2, axis=1)
    scale = np.mean(np.diff(sales, axis=1) ** 2, axis=1)
    rmsse = np.sqrt(squared_errors / (scale + 1e-10))

    total_revenue = np.nansum(sales[:, -28:] * daily_prices[:, -28:], axis=1) 
    weight = total_revenue / np.nansum(total_revenue) 

    wrmsse = np.nansum(weight * rmsse)
    
    return wrmsse

In [ ]:
calculate_wrmsse(test_labels, test_forecasts) # 12

In [ ]:
from tqdm import tqdm
from scipy.linalg import pinv

def create_S(y_id):
    sales = pd.read_csv('../data/original/sales_train_validation.csv')
    sales['id'] = sales['id'].str.replace('_validation', '') # 30490

    states = sales['state_id'].unique()  
    stores = sales['store_id'].unique()
    cats = sales['cat_id'].unique()
    depts = sales['dept_id'].unique()
    states_cats = [f"{state}_{cat}" for state in states for cat in cats]
    states_depts = [f"{state}_{dept}" for state in states for dept in depts]
    stores_cats = [f"{store}_{cat}" for store in stores for cat in cats]
    stores_depts = [f"{store}_{dept}" for store in stores for dept in depts]
    items = sales['item_id'].unique()
    items_states = [f"{item}_{state}" for item in items for state in states]
    items_stores = [f"{item}_{store}" for item in items for store in stores]

    S = np.zeros((42840, 30490))

    for i, id in tqdm(enumerate(y_id), total=len(y_id)):
        if id == 'total':
            S[0, :] = 1            
        elif id in states:
            S[i, :] = sales['id'].isin(sales[sales['state_id'] == id]['id']).astype(int).values
        elif id in stores:
            S[i, :] = sales['id'].isin(sales[sales['store_id'] == id]['id']).astype(int).values
        elif id in cats:
            S[i, :] = sales['id'].isin(sales[sales['cat_id'] == id]['id']).astype(int).values
        elif id in depts:
            S[i, :] = sales['id'].isin(sales[sales['dept_id'] == id]['id']).astype(int).values
        elif id in states_cats:
            state, cat = id.split('_')
            S[i, :] = sales['id'].isin(sales[(sales['state_id'] == state) & (sales['cat_id'] == cat)]['id']).astype(int).values
        elif id in states_depts:
            splitted_id = id.split('_')
            state, dept = splitted_id[0], '_'.join(splitted_id[1:])
            S[i, :] = sales['id'].isin(sales[(sales['state_id'] == state) & (sales['dept_id'] == dept)]['id']).astype(int).values
        elif id in stores_cats:
            splitted_id = id.split('_')
            store, cat = '_'.join(splitted_id[:2]), splitted_id[2]
            S[i, :] = sales['id'].isin(sales[(sales['store_id'] == store) & (sales['cat_id'] == cat)]['id']).astype(int).values
        elif id in stores_depts:
            splitted_id = id.split('_')
            store, dept = '_'.join(splitted_id[:2]), '_'.join(splitted_id[2:])
            S[i, :] = sales['id'].isin(sales[(sales['store_id'] == store) & (sales['dept_id'] == dept)]['id']).astype(int).values
        elif id in items:
            S[i, :] = sales['id'].isin(sales[sales['item_id'] == id]['id']).astype(int).values
        elif id in items_states:
            splitted_id = id.split('_')
            item, state = '_'.join(splitted_id[:3]), '_'.join(splitted_id[3:])
            S[i, :] = sales['id'].isin(sales[(sales['item_id'] == item) & (sales['state_id'] == state)]['id']).astype(int).values
        elif id in items_stores:
            splitted_id = id.split('_')
            item, store = '_'.join(splitted_id[:3]), '_'.join(splitted_id[3:])
            S[i, :] = sales['id'].isin(sales[(sales['item_id'] == item) & (sales['store_id'] == store)]['id']).astype(int).values
        else:
            print(f"Error: {id} not found")

    return S

def compute_W(y_actual, y_pred):
    E = y_actual - y_pred
    W = (1 / (E.shape[1] - 1)) * (E @ E.T)
    return W

S = create_S(test_ids)
W = compute_W(test_labels, test_forecasts)

In [4]:
from scipy.linalg import inv

def convert_to_appropriate_dtype(matrix):
    c_min = matrix.min()
    c_max = matrix.max()
    
    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
        return matrix.astype(np.float16)
    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
        return matrix.astype(np.float32)
    else:
        return matrix.astype(np.float64)

S = convert_to_appropriate_dtype(S)
W = convert_to_appropriate_dtype(W)
test_forecasts = convert_to_appropriate_dtype(test_forecasts)

W_inv = inv(W)
ST_Winv = S.T @ W_inv
ST_Winv_S = ST_Winv @ S
ST_Winv_S_inv = inv(ST_Winv_S)
S_ST_Winv_S_inv = S @ ST_Winv_S_inv

In [ ]:
import os
import numpy as np
from joblib import Parallel, delayed

def chunked_dot_memmap(A, B, chunk_size, n_jobs=-1, temp_dir='C:/temp'):
    def process_chunk_memmap(start, end, A, B, result_memmap):
        result_memmap[start:end] = A[start:end] @ B

    n_chunks = (A.shape[0] + chunk_size - 1) // chunk_size
    result_shape = (A.shape[0], B.shape[1])
    
    # Ensure the temporary directory exists
    os.makedirs(temp_dir, exist_ok=True)
    
    # Use an absolute path for the memory-mapped file
    result_memmap_path = os.path.join(temp_dir, 'result_memmap.dat')
    result_memmap = np.memmap(result_memmap_path, dtype=A.dtype, mode='w+', shape=result_shape)

    Parallel(n_jobs=n_jobs)(
        delayed(process_chunk_memmap)(i * chunk_size, min((i + 1) * chunk_size, A.shape[0]), A, B, result_memmap)
        for i in range(n_chunks)
    )

    return result_memmap

# Example usage
# Assuming S_ST_Winv_S_inv and ST_Winv are already defined
chunk_size = 1000  # Adjust chunk size based on available memory
P = chunked_dot_memmap(S_ST_Winv_S_inv, ST_Winv, chunk_size=chunk_size)

# Use the result
print("Shape of P:", P.shape)

In [ ]:
test_forecasts_reconciled = P @ test_forecasts

print("Before reconciliation:", test_forecasts[:10])
print("After reconciliation:", test_forecasts_reconciled[:10])

In [ ]:
def mint_reconciliation(S, W, y_pred):
    W_inv = inv(W)
    P = S @ inv(S.T @ W_inv @ S) @ S @ W_inv
    return P @ y_pred

test_forecasts_reconciled = mint_reconciliation(S, W, test_forecasts)

print("Before reconcilation:", test_forecasts[:10])
print("After reconcilation:", test_forecasts_reconciled[:10])

In [ ]:
def calculate_wrmsse(y_true, y_pred):
    sales = pd.read_csv('../data/original/sales_train_validation.csv')
    sell_prices = pd.read_csv('../data/original/sell_prices.csv')

    sales = sales.iloc[:, 6:].values

    sell_prices['id'] = sell_prices['store_id'] + '_' + sell_prices['item_id']
    sell_prices = sell_prices[sell_prices['wm_yr_wk'] <= 11613]
    sell_prices = sell_prices.pivot(index='id', columns='wm_yr_wk', values='sell_price')
    sell_prices = sell_prices.values

    N, h = y_true.shape 
    w = sell_prices.shape[1]  

    daily_prices = np.repeat(sell_prices, repeats=7, axis=1)[:, -sales.shape[1]:]
    daily_prices = np.where(np.isnan(daily_prices), np.nan, daily_prices)
    
    squared_errors = np.mean((y_true - y_pred) ** 2, axis=1)
    scale = np.mean(np.diff(sales, axis=1) ** 2, axis=1)
    rmsse = np.sqrt(squared_errors / (scale + 1e-10))

    total_revenue = np.nansum(sales[:, -28:] * daily_prices[:, -28:], axis=1) 
    weight = total_revenue / np.nansum(total_revenue) 

    wrmsse = np.nansum(weight * rmsse)
    
    return wrmsse

calculate_wrmsse(test_labels, test_forecasts) # 12